<h1>logistic regression</h1>
# setting training set labels to 50/50 %
    <p>90 class 1 samples and 90 class 0 samples by record reduction

# <h4>score: private 0.618 ; public 0.656</h4>

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures


# df = pd.read_csv('./train.csv', index_col = True)
url = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/train.csv'
df = pd.read_csv(url, index_col="id")
df['target'] = df['target'].astype(int)

parameters_dictionary = {}

# print(df.groupby('target'))

df_class_1 = df[df['target'] == 1]
df_class_0 = df[df['target'] == 0]

print(len(df_class_1), ":", len(df_class_0))

df_class_1 = df_class_1[:-70]


print(len(df_class_1), ":", len(df_class_0))

df = pd.concat([df_class_0, df_class_1])

for element in df:
    if element != "target":
        parameters_dictionary.update({element : df[element]})



label_dictionary = {'label': df['target']}

df_X = pd.DataFrame(data = parameters_dictionary)
df_Y = pd.DataFrame(data = label_dictionary)




# print(df_Y.groupby('label').size())



160 : 90
90 : 90


In [15]:
# implement logistic regression


###
#   function: predict_logistic_regression
#       @param: input_x => training dataset features
#       @param: input_y => training dataset label
#       @param: input_z => testing dataset records (features only)
#   
#   output: array/list of predicted result of input_z
###
model = LogisticRegression(max_iter=10000)

def predict_logistic_regression(input_x, input_y, input_z):
    # build the model based on training data
    model.fit(input_x, input_y)

    # insert testing dataset to get the predicted output
    predicted_values = model.predict(input_z)

    return predicted_values
    


In [16]:
#  Applying 5-fold-cross-validation
k = 5
kf = KFold(n_splits = k, random_state = None)


# A function to run 5-fold cross validation on polynomial_logistic_regression
def kFold_logistic_regression(input_X, input_Y):

    validation_f1_score = []

    for train_index , validation_index in kf.split(input_X):

        # print('Start model fitting, k=', train_index[0], ' - ', train_index[-1], ' ...')

        X_train , X_valid = input_X.iloc[train_index,:], input_X.iloc[validation_index,:]
        y_train, y_valid = input_Y.iloc[train_index], input_Y.iloc[validation_index]

        # print('X_train: ', X_train)

        y_train = y_train.to_numpy().ravel()
        y_valid = y_valid.to_numpy().ravel()
        
        pred_testValues = predict_logistic_regression(X_train, y_train, X_valid)

        # count the f1 score(true records / predicted records)
        valid_f1 = f1_score(y_valid, pred_testValues)
        validation_f1_score.append(valid_f1)

    return validation_f1_score


In [17]:
testing_f1 = []

testing_f1_score = kFold_logistic_regression(df_X, df_Y)
testing_f1.append(testing_f1_score)


# print('testing set F1 score (poly 2 to 12) under 5-fold-cross-validation: ')
for row in testing_f1:
    print(row)

[0.0, 0.0, 0.7027027027027027, 0.6153846153846153, 0.6909090909090909]


<h3>Real prediction on testing data</h3>

In [18]:
import csv


url_test = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/test.csv'
df_test = pd.read_csv(url_test, index_col="id")

# print(df_test)

pred_values = predict_logistic_regression(df_X, df_Y, df_test)



with open('./result_folder/logisticV3.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "target"])
    for i in range(250, 20000):
        writer.writerow([i, pred_values[i-250]])
    file.close()


c:\Python39\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
